In [21]:
import pandas as pd
import json

lines = pd.read_csv("../data/lines/strikeouts.csv")
lines.head(99)

,player,k_line,over_american_price,over_decimal_price,over_payout_multiplier,under_american_price,under_decimal_price,under_payout_multiplier
0,Nick Lodolo,4.5,-129,1.78,1.00,-104,1.97,1.00
1,Gunnar Hoglund,3.5,-115,1.87,1.00,-115,1.87,1.00
2,Sonny Gray,6.5,121,2.21,1.14,-162,1.62,0.81
3,Chad Patrick,4.5,122,2.22,1.15,-162,1.62,0.81
4,Jesús Luzardo,6.5,111,2.11,1.11,-148,1.68,0.87
5,Davis Martin,3.5,-115,1.87,1.00,-115,1.87,1.00
6,Kodai Senga,6.5,-159,1.63,0.82,119,2.19,1.13
7,Antonio Senzatela,3.5,131,2.31,1.17,-176,1.57,0.77
8,Kyle Hendricks,3.5,-102,1.99,1.05,-132,1.76,0.91
9,Edward Cabrera,5.5,-103,1.98,1.04,-130,1.77,0.92


In [22]:

probables = lines['player'].unique().tolist()

df_2025 = pd.read_parquet("../data/processed/pitcher_game_data_2025.parquet")
df_latest = (
    df_2025[df_2025['pitcher_name'].isin(probables)]
    .sort_values(["pitcher_name", 'game_date'])
    .groupby("pitcher_name")
    .tail(1)
)

with open("../models/XGB_Tuned.json") as f:
    meta = json.load(f)

X_pred = df_latest[meta["features"]]

In [23]:
from joblib import load
model = load("../models/xgb_tuned_pitcher_k_model.joblib")
df_latest['model_k_pred'] = model.predict(X_pred)

In [25]:
merged = df_latest.merge(lines, left_on='pitcher_name', right_on='player', how='left')
merged['edge'] = merged['model_k_pred'] - merged['k_line']
picks = merged[['pitcher_name', 'k_line', 'model_k_pred', 'edge']]
picks.head(99)

,pitcher_name,k_line,model_k_pred,edge
0,Antonio Senzatela,3.5,3.613424,0.113424
1,Brandon Pfaadt,4.5,3.912566,-0.587434
2,Bryce Miller,4.5,4.505109,0.005109
3,Davis Martin,3.5,4.730012,1.230012
4,Dylan Cease,7.5,5.103074,-2.396926
5,Edward Cabrera,5.5,5.654613,0.154613
6,Kodai Senga,6.5,5.964560,-0.535440
7,Kyle Hendricks,3.5,4.429575,0.929575
8,Nick Lodolo,4.5,4.822745,0.322745
9,Robbie Ray,6.5,5.512727,-0.987273


In [26]:
# Absolute edge and sort
merged['abs_edge'] = merged['edge'].abs()
top = merged.sort_values('abs_edge', ascending=False).reset_index(drop=True)

# Keep the useful fields
top_picks = top[['pitcher_name', 'k_line', 'model_k_pred', 'edge', 'abs_edge']]


In [28]:
top_picks.head(5)

,pitcher_name,k_line,model_k_pred,edge,abs_edge
0,Dylan Cease,7.5,5.103074,-2.396926,2.396926
1,Sonny Gray,6.5,4.692000,-1.808000,1.808000
2,Davis Martin,3.5,4.730012,1.230012,1.230012
3,Robbie Ray,6.5,5.512727,-0.987273,0.987273
4,Kyle Hendricks,3.5,4.429575,0.929575,0.929575
